In [ ]:
!pip install ultralytics flask flask-cors pyngrok yt_dlp ffmpeg-python
from flask import Flask, request, jsonify, send_file, Response
from flask_cors import CORS
from ultralytics import YOLO
import cv2
import numpy as np
import base64
import yt_dlp
import traceback, tempfile, os, ffmpeg, shutil

## 1. Loading the YOLOv8 Model

In [ ]:
def load_model(model_path):
    """
    Loads a YOLO object detection model from the specified model_path.

    Parameters:
        model_path (str): The path to the YOLO model file.

    Returns:
        A YOLO object detection model.
    """
    model = YOLO(model_path)
    return model

In [ ]:
from ultralytics import YOLO
yolo_path = "/content/drive/MyDrive/Colab Notebooks/Object_Detection_App_with_YOLOv8_Streamlit/weights/yolov8n.pt"
model = load_model(yolo_path)

## 2. Processing Images

In [ ]:
def detect_image_bytes(image_bytes, conf=0.4, tracker=None):
    nparr = np.frombuffer(image_bytes, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (720, int(720 * 9 / 16)))

    if tracker:
        result = model.track(image, conf=conf, persist=True, tracker=tracker)
    else:
        result = model.predict(image, conf=conf)

    plotted = result[0].plot()
    _, buffer = cv2.imencode('.jpg', plotted)
    return base64.b64encode(buffer).decode()

## 3. Processing Videos Youtube through URLs

In [ ]:
def move_moov_atom_to_front(input_path):
    output_path = input_path.replace(".mp4", "_fixed.mp4")
    (
        ffmpeg
        .input(input_path)
        .output(
            output_path,
            vcodec='libx264',      # Encode sang H.264 chuẩn
            crf=23,                # Chất lượng (thấp hơn = đẹp hơn, file to hơn)
            preset='veryfast',     # Tốc độ encode
            movflags='faststart',  # Cho phép streaming
            pix_fmt='yuv420p'      # Tương thích HTML5 player
        )
        .run(overwrite_output=True)
    )
    return output_path

def detect_youtube(youtube_url, conf=0.4, tracker=None):
    temp_dir = tempfile.mkdtemp()
    temp_download_path = os.path.join(temp_dir, "downloaded.mp4")

    ydl_opts = {
        'format': 'best[ext=mp4]',
        'quiet': True,
        'no_warnings': True,
        'noplaylist': True,
        'outtmpl': temp_download_path,
        'http_headers': {'User-Agent': 'Mozilla/5.0'}
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([youtube_url])
        except yt_dlp.utils.DownloadError as e:
            msg = str(e).lower()
            if any(key in msg for key in [
                "members-only", "private", "sign in", "age-restricted", "join this channel"
            ]):
                raise PermissionError("🔒 Video is private or members-only. Please use a public link.")
            elif any(key in msg for key in ["not found", "unavailable", "url error", "unable to download"]):
                raise ValueError("❌ Invalid or unavailable YouTube link.")
            else:
                raise RuntimeError(f"Unable to process YouTube video: {e}")

    cap = cv2.VideoCapture(temp_download_path)
    if not cap.isOpened():
        shutil.rmtree(temp_dir, ignore_errors=True)
        raise RuntimeError("Cannot open downloaded YouTube video.")

    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    width = 720
    height = int(720 * 9 / 16)

    temp_video_path = os.path.join(temp_dir, "processed.mp4")
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(temp_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (width, height))
        if tracker:
            result = model.track(frame, conf=conf, persist=True, tracker=tracker)
        else:
            result = model.predict(frame, conf=conf)
        plotted = result[0].plot()
        out.write(plotted)

    cap.release()
    out.release()

    fixed_path = move_moov_atom_to_front(temp_video_path)

    # Tạo file kết quả ở thư mục tạm hệ thống (ngoài temp_dir)
    final_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name
    shutil.move(fixed_path, final_path)

    # Xóa thư mục tạm
    shutil.rmtree(temp_dir, ignore_errors=True)

    return final_path

## 4. Real-Time Webcam Detection

In [ ]:
def detect_webcam(conf=0.4, tracker=None, max_frames=100):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Cannot open webcam")

    frames = []
    count = 0

    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (720, int(720 * 9 / 16)))

        if tracker:
            result = model.track(frame, conf=conf, persist=True, tracker=tracker)
        else:
            result = model.predict(frame, conf=conf)

        plotted = result[0].plot()
        _, buffer = cv2.imencode('.jpg', plotted)
        frames.append(base64.b64encode(buffer).decode())

        count += 1

    cap.release()
    return frames

## 5. Process video uploaded

In [ ]:
def detect_video_bytes(video_bytes, conf=0.4, tracker=None):
    import uuid
    import subprocess

    # Lưu video gốc tạm thời
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as temp_vid:
        temp_vid.write(video_bytes)
        temp_vid_path = temp_vid.name

    cap = cv2.VideoCapture(temp_vid_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = 720
    height = int(720 * 9 / 16)

    # Tạo file output video
    output_filename = f"/tmp/output_{uuid.uuid4().hex}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264
    out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (width, height))

        if tracker:
            result = model.track(frame, conf=conf, persist=True, tracker=tracker)
        else:
            result = model.predict(frame, conf=conf)

        plotted = result[0].plot()
        out.write(plotted)

    cap.release()
    out.release()
    os.remove(temp_vid_path)

    # Move moov atom to start
    fixed_path = move_moov_atom_to_front(output_filename)
    os.remove(output_filename)

    return fixed_path  # Trả về path file video

## Flask API

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/')
def index():
    return {'message': 'YOLOv8 backend is running.'}

@app.route('/detect/image', methods=['POST'])
def detect_image():
    file = request.files.get('image')
    if not file:
        return jsonify({'error': 'No image file provided'}), 400

    conf = float(request.form.get('conf', 0.4))
    tracker = request.form.get('tracker', None)

    try:
        result_b64 = detect_image_bytes(file.read(), conf=conf, tracker=tracker)
        return jsonify({'result': result_b64})
    except Exception as e:
        print("Error when processing images:", e)
        traceback.print_exc()  # In ra traceback chi tiết
        return jsonify({'error': str(e)}), 500

@app.route('/detect/youtube', methods=['POST'])
def detect_youtube_api():
    data = request.get_json()
    url = data.get('url', '').strip()
    conf = float(data.get('conf', 0.4))
    tracker = data.get('tracker') or None

    if not url or not url.startswith("http"):
        return jsonify({'error': "❌ Please provide a valid YouTube URL."}), 400

    try:
        output_path = detect_youtube(url, conf=conf, tracker=tracker)
        return send_file(output_path, mimetype="video/mp4")
    except PermissionError as e:
        return jsonify({'error': str(e)}), 401
    except ValueError as e:
        return jsonify({'error': str(e)}), 400
    except Exception as e:
        traceback.print_exc()
        return jsonify({'error': f"YouTube processing error: {e}"}), 500

@app.route('/detect/video', methods=['POST'])
def detect_video_api():
    file = request.files.get('video')

    try:
        conf = float(request.form.get('conf', 0.4))
    except (ValueError, TypeError):
        conf = 0.4

    tracker = request.form.get('tracker')
    if tracker == "":
        tracker = None

    try:
        output_path = detect_video_bytes(file.read(), conf=conf, tracker=tracker)
        # return send_file(output_path, mimetype="video/mp4", as_attachment=True, download_name="detected_video.mp4")
        return send_file(output_path, mimetype="video/mp4")
    except Exception as e:
        print("Error when processing video:", e)
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

@app.route('/detect/frame', methods=['POST'])
def detect_frame_api():
    try:
        file = request.files['frame']
        conf = float(request.form.get('conf', 0.4))
        tracker = request.form.get('tracker', None)

        img_bytes = file.read()
        np_img = np.frombuffer(img_bytes, np.uint8)
        frame = cv2.imdecode(np_img, cv2.IMREAD_COLOR)

        if tracker:
            result = model.track(frame, conf=conf, persist=True, tracker=tracker)
        else:
            result = model.predict(frame, conf=conf)

        plotted = result[0].plot()
        _, buffer = cv2.imencode('.jpg', plotted)
        return Response(buffer.tobytes(), mimetype='image/jpeg')

    except Exception as e:
        return jsonify({'error': str(e)}), 500

## Ngrok Export Port

In [ ]:
# Đọc token từ file (và không hiển thị ra output)
with open('/content/drive/MyDrive/Colab Notebooks/Object_Detection_App_with_YOLOv8_Streamlit/ngrok_token.txt', 'r') as f:
    NGROK_AUTH_TOKEN = f.read().strip()
!ngrok config add-authtoken {NGROK_AUTH_TOKEN}


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# !ngrok http 8501 > /dev/null 2>&1 &
!pkill -f ngrok
!pkill -f flask

0: 384x640 6 cars, 256.0ms
Speed: 3.7ms preprocess, 256.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)



In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("YOLO API public URL:", public_url)
app.run(port=5000)

YOLO API public URL: NgrokTunnel: "https://1601ccc832df.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit



0: 480x640 1 person, 218.6ms
Speed: 4.0ms preprocess, 218.6ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)


INFO:werkzeug:127.0.0.1 - - [08/Aug/2025 08:15:14] "POST /detect/frame HTTP/1.1" 200 -



0: 480x640 1 person, 1 bicycle, 173.5ms
Speed: 2.9ms preprocess, 173.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)


...

INFO:werkzeug:127.0.0.1 - - [08/Aug/2025 08:50:33] "POST /detect/frame HTTP/1.1" 200 -



0: 480x640 1 person, 1 bicycle, 169.9ms
Speed: 3.2ms preprocess, 169.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)


INFO:werkzeug:127.0.0.1 - - [08/Aug/2025 08:50:34] "POST /detect/frame HTTP/1.1" 200 -



0: 480x640 1 person, 1 bicycle, 176.6ms
Speed: 2.9ms preprocess, 176.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)


INFO:werkzeug:127.0.0.1 - - [08/Aug/2025 08:50:35] "POST /detect/frame HTTP/1.1" 200 -
